# CROSS VALIDATION

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../DATA/Advertising.csv")

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9



## Train | Test Split Procedure 

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Test Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Test Data (by creating predictions and comparing to Y_test)
7. Adjust Parameters as Necessary and repeat steps 5 and 6

0. Clean and adjust data as necessary for X and y

In [4]:
X = df.drop('sales',axis=1)

In [5]:
y = df['sales']

1. Split Data in Train/Test for both X and y

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

2. Fit/Train Scaler on Training X Data

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()

In [10]:
scaler.fit(X_train)

StandardScaler()

In [11]:
X_train = scaler.transform(X_train)

3. Scale X Test Data

In [12]:
X_test = scaler.transform(X_test)

4. Create Model

In [13]:
from sklearn.linear_model import Ridge

In [14]:
model = Ridge(alpha=100)

5. Fit/Train Model on X Train Data

In [15]:
model.fit(X_train,y_train)

Ridge(alpha=100)

6. Evaluate Model on X Test Data (by creating predictions and comparing to Y_test)

In [16]:
y_pred = model.predict(X_test)

In [17]:
from sklearn.metrics import mean_squared_error

In [18]:
mean_squared_error(y_test,y_pred)

7.341775789034129

7. Adjust Parameters as Necessary and repeat steps 5 and 6

In [19]:
model_two = Ridge(alpha=1)

In [20]:
model_two.fit(X_train,y_train)

Ridge(alpha=1)

In [21]:
y_pred_two = model_two.predict(X_test)

In [22]:
mean_squared_error(y_test,y_pred_two)

2.3190215794287514

## Train | Validation | Test Split Procedure

This is often also called a "hold-out" set, since you should not adjust parameters based on the final test set, but instead use it *only* for reporting final expected performance.

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Validation/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Eval Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)
7. Adjust Parameters as Necessary and repeat steps 5 and 6
8. Get final metrics on Test set (not allowed to go back and adjust after this!)

0. Clean and adjust data as necessary for X and y

In [23]:
X = df.drop('sales',axis=1)
y = df['sales']

1. Split Data in Train/Validation/Test for both X and y

In [24]:
from sklearn.model_selection import train_test_split

# 70% of data is training data, set aside other 30%
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)

# Remaining 30% is split into evaluation and test sets
# Each is 15% of the original data size
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

2. Fit/Train Scaler on Training X Data

In [25]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

3. Scale X Eval Data

In [26]:
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

4. Create Model

In [27]:
from sklearn.linear_model import Ridge

In [28]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)

5. Fit/Train Model on X Train Data

In [29]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [30]:
y_eval_pred = model.predict(X_eval)

6. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)

In [31]:
from sklearn.metrics import mean_squared_error

In [32]:
mean_squared_error(y_eval,y_eval_pred)

7.320101458823872

7. Adjust Parameters as Necessary and repeat steps 5 and 6

In [33]:
model = Ridge(alpha=1)

In [34]:
model.fit(X_train,y_train)

Ridge(alpha=1)

In [35]:
y_eval_pred = model.predict(X_eval)

In [36]:
mean_squared_error(y_eval,y_eval_pred)

2.3837830750569866

8. Get final metrics on Test set (not allowed to go back and adjust after this!)

In [37]:
y_final_test_pred = model.predict(X_test)

In [38]:
mean_squared_error(y_test,y_final_test_pred)

2.254260083800517

## Cross Validation with cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [39]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [40]:
model = Ridge(alpha=100)

In [41]:
from sklearn.model_selection import cross_val_score

In [42]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [43]:
# 5 errors (cv or K-fold = 5)
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [44]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

8.215396464543607

### Adjust model based on metrics

In [45]:
model = Ridge(alpha=1)

In [46]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [47]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

3.344839296530695

### Final Evaluation (Can no longer edit parameters after this!)

In [48]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [49]:
y_final_test_pred = model.predict(X_test)

In [50]:
mean_squared_error(y_test,y_final_test_pred)

2.3190215794287514

# Cross Validation with cross_validate


In [52]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [53]:
model = Ridge(alpha=100)

In [54]:
from sklearn.model_selection import cross_validate

In [55]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [56]:
scores

{'fit_time': array([0.00199914, 0.00099921, 0.00099969, 0.        , 0.00100183]),
 'score_time': array([0.00100183, 0.        , 0.        , 0.00099921, 0.        ]),
 'test_neg_mean_absolute_error': array([-2.31243044, -1.74653361, -2.56211701, -2.01873159, -2.27951906]),
 'test_neg_mean_squared_error': array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
         -8.38562723]),
 'test_max_error': array([ -6.44988486,  -5.58926073, -10.33914027,  -6.61950405,
         -7.75578515])}

In [57]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.001999,0.001002,-2.312430,-9.325530,-6.449885
1,0.000999,0.000000,-1.746534,-4.944962,-5.589261
2,0.001000,0.000000,-2.562117,-11.396652,-10.339140
3,0.000000,0.000999,-2.018732,-7.024211,-6.619504
4,0.001002,0.000000,-2.279519,-8.385627,-7.755785


In [58]:
pd.DataFrame(scores).mean()

fit_time                        0.001000
score_time                      0.000400
test_neg_mean_absolute_error   -2.183866
test_neg_mean_squared_error    -8.215396
test_max_error                 -7.350715
dtype: float64

### Adjust model based on metrics

In [60]:
model = Ridge(alpha=1)

In [61]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [62]:
pd.DataFrame(scores).mean()

fit_time                        0.000801
score_time                      0.000599
test_neg_mean_absolute_error   -1.319685
test_neg_mean_squared_error    -3.344839
test_max_error                 -5.161145
dtype: float64

### Final Evaluation (Can no longer edit parameters after this!)

In [63]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [64]:
y_final_test_pred = model.predict(X_test)

In [65]:
mean_squared_error(y_test,y_final_test_pred)

2.3190215794287514